In [2]:
import gc
import pandas as pd

In [3]:
import os
# Get the number of CPU cores
num_cores = os.cpu_count()
max_proc = num_cores * 2 + 1
# Print the result
print(f"Number of CPU cores: {num_cores}, max number of IO processes: {max_proc}")

Number of CPU cores: 8, max number of IO processes: 17


In [4]:
from multiprocessing import Pool

def f(chunk):
    gc.collect()
    return pd.json_normalize(chunk.to_dict('records'))

# Open VT reports and convert it to a DataFrame
def extract_reports(file_path) -> pd.DataFrame:
    chunks = pd.read_json(file_path, lines=True, orient='records', chunksize=100)
    return pd.concat([
        f(chunk) for chunk in chunks for _, report in chunk.iterrows()
    ])

# def extract_reports(file_path) -> pd.DataFrame:
#     chunks = [report for chunk in pd.read_json(file_path, lines=True, orient='records', chunksize=1000) 
#               for _, report in chunk.iterrows()]
#     chunks = chunks[:3]
#     with Pool(5) as p:
#         return pd.concat(p.map(lambda report: pd.json_normalize(report), chunks))
#     

In [ ]:
reports_path = './vt_reports/vt_reports67k.jsons'
reports = extract_reports(file_path=reports_path).head()

In [ ]:
def extract_malware_family(file_path) -> pd.DataFrame:
    df = pd.read_csv(file_path, usecols=['SHA256', 'FAMILY'])
    return df.rename(str.lower, axis='columns')

In [8]:
malware_families_path = "./vt/siggregator_all_samples_no_fuzzy_hash.csv"
df_malware_families = extract_reports(file_path=malware_families_path)
df_malware_families.head()

ValueError: Expected object or value

In [6]:
# Join dataframes
df = pd.merge(left=df_malwares_first_sub_date, right=df_malware_families, on='sha256')
df.head()

,sha256,first_submission_date,family
0,98f8e26e12b978102fa39c197f300ebe5fe535617737d5...,1630575593,tnega
1,7b2999ffadbc3b5b5c5e94145ca4e2f8de66ac1e3ddd52...,1629375559,quasar
2,e7569d494fe00be04ef6c9fcc5e54720c0df623b08e79d...,1362057319,pasta
3,1ed60c04f572b6acb9f64c31db55ef5c6b5465bd4da1eb...,1630624233,cjishu
4,4c4aaff20a57213d9a786e56ad22f1eaa94694a2f1042b...,1592186154,kingsoft


In [9]:
df.to_csv("merge.csv", index=False)